In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
%run split_data.ipynb

In [ ]:
num_classes = 105
criterion = nn.CrossEntropyLoss()
learning_rates = [0.001, 0.004]
num_epochs = 30
optimizers = []
all_history = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_directory = 'trening'
validate_directory = 'validate'

In [ ]:
def train_model(lr):

    resnet = models.resnet50(pretrained=True)


    for param in resnet.parameters():
        param.requires_grad = False

    
    for param in resnet.layer4.parameters():
        param.requires_grad = True

    for param in resnet.layer3.parameters():
        param.requires_grad = True
    
    for param in resnet.layer2.parameters():
        param.requires_grad = True

    resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

    resnet.to(device)

    train_loader = load_images_and_create_dataloader(train_directory, batch_size=32)

    valid_loader = load_images_and_create_dataloader(validate_directory, batch_size=32)

    optimizer = optim.SGD([
        {'params': resnet.layer2.parameters(), 'lr': lr / 5},
        {'params': resnet.layer3.parameters(), 'lr': lr / 2},
        {'params': resnet.layer4.parameters(), 'lr': lr}
    ], lr=lr, momentum=0.9)
    
    optimizers.append(optimizer)
    
    train_loss_history = []
    valid_loss_history = []
    train_acc_history = []
    valid_acc_history = []

    for epoch in range(num_epochs):
        resnet.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = resnet(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total

        train_loss_history.append(epoch_loss)
        train_acc_history.append(epoch_acc.item())

        resnet.eval()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = resnet(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total += labels.size(0)

            epoch_loss = running_loss / total
            epoch_acc = running_corrects.double() / total

            valid_loss_history.append(epoch_loss)
            valid_acc_history.append(epoch_acc.item())

        print(f'Epoch {epoch}/{num_epochs - 1}')
        print(f'Train Loss: {train_loss_history[-1]:.4f} Acc: {train_acc_history[-1]:.4f}')
        print(f'Valid Loss: {valid_loss_history[-1]:.4f} Acc: {valid_acc_history[-1]:.4f}')

    all_history.append((train_loss_history, valid_loss_history, train_acc_history, valid_acc_history))

In [ ]:
for lr in learning_rates:
    train_model(lr)

In [ ]:
plt.figure(figsize=(10, 5))


for i, lr in enumerate(learning_rates):
    train_loss, valid_loss, _, _ = all_history[i]
    plt.plot(range(num_epochs), train_loss, label=f'Training Loss (lr={lr})')
    plt.plot(range(num_epochs), valid_loss, label=f'Validation Loss (lr={lr})')


plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss for Different Learning Rates Resnet')


plt.savefig('training_validation_loss_resnet_all_lr.png')
plt.show()


In [ ]:
plt.figure(figsize=(10, 5))


for i, lr in enumerate(learning_rates):
    _, _, train_acc, valid_acc = all_history[i]
    plt.plot(range(num_epochs), train_acc, label=f'Training Acc (lr={lr})')
    plt.plot(range(num_epochs), valid_acc, label=f'Validation Acc (lr={lr})')


plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy for Different Learning Rates Resnet')


plt.savefig('training_validation_accuracy_resnet_all_lr.png')
plt.show()
